In [74]:
import pandas as pd
import numpy as np
import string
import re
import nltk

In [75]:
df = pd.read_csv('DataScientist.csv')
df = df.drop("Job Description", axis=1)
df = df.drop("Unnamed: 0", axis=1)


In [76]:
for i in df.index:
    if df.loc[i,"Easy Apply"] == "-1":
        df.loc[i,"Easy Apply"] = False
df["Easy Apply"]=df["Easy Apply"].astype(bool)
print(df["Easy Apply"].to_string())




0       False
1       False
2        True
3       False
4       False
5       False
6       False
7        True
8        True
9       False
10       True
11      False
12      False
13      False
14      False
15       True
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
30      False
31       True
32      False
33      False
34       True
35      False
36      False
37      False
38      False
39      False
40      False
41      False
42       True
43      False
44      False
45      False
46      False
47      False
48      False
49       True
50      False
51      False
52      False
53      False
54      False
55      False
56      False
57      False
58      False
59      False
60      False
61      False
62      False
63      False
64      False
65      False
66       True
67      False
68       True
69      False
70       True
71    

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              3909 non-null   int64  
 1   Job Title          3909 non-null   object 
 2   Salary Estimate    3909 non-null   object 
 3   Rating             3909 non-null   float64
 4   Company Name       3909 non-null   object 
 5   Location           3909 non-null   object 
 6   Headquarters       3909 non-null   object 
 7   Size               3909 non-null   object 
 8   Founded            3909 non-null   int64  
 9   Type of ownership  3909 non-null   object 
 10  Industry           3909 non-null   object 
 11  Sector             3909 non-null   object 
 12  Revenue            3909 non-null   object 
 13  Competitors        3909 non-null   object 
 14  Easy Apply         3909 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(11)
memory usage: 431.5+ KB


In [78]:
#data cleaning
df.replace(['-1'], [np.nan], inplace=True)
df.replace(['-1.0'], [np.nan], inplace=True)
df.replace([-1], [np.nan], inplace=True)

In [79]:
df.isnull().sum()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              3909 non-null   int64  
 1   Job Title          3909 non-null   object 
 2   Salary Estimate    3909 non-null   object 
 3   Rating             3500 non-null   float64
 4   Company Name       3909 non-null   object 
 5   Location           3909 non-null   object 
 6   Headquarters       3669 non-null   object 
 7   Size               3680 non-null   object 
 8   Founded            2932 non-null   float64
 9   Type of ownership  3680 non-null   object 
 10  Industry           3363 non-null   object 
 11  Sector             3363 non-null   object 
 12  Revenue            3680 non-null   object 
 13  Competitors        1149 non-null   object 
 14  Easy Apply         3909 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(11)
memory usage: 431.5+ KB


In [80]:
#seperating salary estimate into min and max salary 
df_salary = df["Salary Estimate"].str.split("-",expand=True)
minimum_salary = df_salary[0]
minimum_salary = minimum_salary.str.replace("K"," ")
maximum_salary = df_salary[1].str.replace('(Glassdoor est.)', ' ')
maximum_salary = maximum_salary.str.replace('(', ' ')
maximum_salary = maximum_salary.str.replace(')', ' ')
maximum_salary = maximum_salary.str.replace('K', ' ')
maximum_salary = maximum_salary.str.replace('Employer est.', ' ')
maximum_salary = maximum_salary.str.replace('Per Hour', ' ')

maximum_salary = maximum_salary.str.replace('$', ' ').fillna(0).astype(int)
minimum_salary = minimum_salary.str.replace('$', ' ').fillna(0).astype(int)





/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/4083596630.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  maximum_salary = df_salary[1].str.replace('(Glassdoor est.)', ' ')
/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/4083596630.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  maximum_salary = maximum_salary.str.replace('(', ' ')
/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/4083596630.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  maximum_salary = maximum_salary.str.replace(')', ' ')
/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/4083596630

In [81]:
df['Minimum Salary'] = minimum_salary
df['Maximum Salary'] = maximum_salary

df.drop('Salary Estimate',axis = 1,inplace = True)
df['Company Name'] = df['Company Name'].str.replace('\n.*', ' ')
df['Est_Salary']= (df['Minimum Salary']+df['Maximum Salary'])/2


/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/2733306494.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Company Name'] = df['Company Name'].str.replace('\n.*', ' ')


In [82]:
#seperating location and HQ

Location = df["Location"].str.split(",",expand=True)
Location_city = Location[0]
Location_state = Location[1]
df["Location City"] = Location_city
df["Location State"] = Location_state
df.drop("Location",axis = 1, inplace=True)

HQ = df["Headquarters"].str.split(",",expand=True)
HQ_city = HQ[0]
HQ_state = HQ[1]
df["HQ City"] = HQ_city
df["HQ State"] = HQ_state
df.drop("Headquarters",axis = 1, inplace=True)



In [83]:
#seperating job tilte from department

department = df["Job Title"].str.split(",", expand=True)
df["Job Title"], df["Department"] = department[0], department[1]
df.drop("Department",axis=1,inplace=True)

In [84]:
df["Job Title"] = df["Job Title"].str.replace(" â","")


In [85]:
df["Revenue"] = df["Revenue"].replace("Unknown / Non-Applicable",np.nan)


In [86]:
df['Revenue'] = df['Revenue'].str.replace('$', ' ')
df['Revenue'] = df['Revenue'].str.replace('(USD)', ' ')
df['Revenue'] = df['Revenue'].str.replace('(', ' ')
df['Revenue'] = df['Revenue'].str.replace(')', ' ')
df['Revenue'] = df['Revenue'].str.replace(' ', '')
df['Revenue'] = df['Revenue'].str.replace('+', '')

/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/2912657363.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Revenue'] = df['Revenue'].str.replace('$', ' ')
/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/2912657363.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Revenue'] = df['Revenue'].str.replace('(USD)', ' ')
/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/2912657363.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Revenue'] = df['Revenue'].str.replace('(', ' ')
/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/2912657363.py:4: FutureWa

In [87]:
df['Revenue'] = df['Revenue'].str.replace('2to5billion', '2billionto5billion')
df['Revenue'] = df['Revenue'].str.replace('5to10billion ', '5billionto10billion')

In [88]:
df['Revenue'] = df['Revenue'].replace('million', ' ')
df['Revenue'] = df['Revenue'].replace('10billion', '10billionto11billion')
df['Revenue'] = df['Revenue'].str.replace('Lessthan1million', '0millionto1million')
df['Revenue'] = df['Revenue'].str.replace('million', ' ')
df['Revenue'] = df['Revenue'].str.replace('billion', '000 ')


In [89]:
df["Revenue"] = df["Revenue"].str.replace(" to","to")


In [90]:
revenue = df["Revenue"].str.split("to",expand=True)

In [91]:
df["max revenue (milions)"] = revenue[1]
df["min revenue (milions)"] = revenue[0]

In [92]:
df.drop("Revenue",axis=1,inplace=True)

In [93]:
df["HQ State"] = df["HQ State"].str.replace("Sweden","SWE")
df["HQ State"] = df["HQ State"].str.replace("Belgium","BE")
df["HQ State"] = df["HQ State"].str.replace("Iran","IR")
df["HQ State"] = df["HQ State"].str.replace("United Kingdom","UK")

In [94]:
df["Founded"] = df["Founded"].astype("Int32")


1999    110
2002    100
2004     91
2015     90
1994     84
       ... 
1792      1
1851      1
1806      1
1843      1
1866      1
Name: Founded, Length: 172, dtype: Int64

In [95]:
df["Job Title"] = df["Job Title"].str.replace("/","|")

In [96]:
df["Company Name"] = df["Company Name"].astype(str)


In [97]:
for i in df.index:
    df.loc[i,"index"] = i
        

In [98]:
df["Type of ownership"] = df["Type of ownership"].str.replace("/","|")

In [99]:
df["Size"] = df["Size"].str.replace("employees","")
df["Size"] = df["Size"].str.replace("+","")
df["Size"] = df["Size"].str.replace(" ","")


/var/folders/4q/b59kyw8x60392hk2b9nn2fqm0000gn/T/ipykernel_11216/350990652.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Size"] = df["Size"].str.replace("+","")


In [100]:
size = df["Size"].str.split("to",expand=True)
df["min_size_employees"], df["max_size_employees"] = size[0], size[1]


In [101]:
df.drop("Size",axis=1,inplace=True)

In [102]:
print(df.to_string())

      index                                                                                                               Job Title  Rating                                           Company Name  Founded               Type of ownership                                  Industry                              Sector                                                                                   Competitors  Easy Apply  Minimum Salary  Maximum Salary  Est_Salary                   Location City   Location State                   HQ City      HQ State max revenue (milions) min revenue (milions) min_size_employees max_size_employees
0         0                                                                                                   Senior Data Scientist     3.5                                                Hopper      2007               Company - Private                           Travel Agencies                    Travel & Tourism                                                    